In [1]:
!featurize dataset download 915bd4c7-559e-4105-a817-f16fa180d350

zsh:1: command not found: featurize


In [2]:
!featurize dataset download 08c66597-d5ed-45f4-be51-59d6ad217913

zsh:1: command not found: featurize


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint


import utils
import models

In [4]:
# embed = utils.Embed(w2v_dic='~/data/merge_sgns_bigram_char300.txt')
embed = utils.Embed(w2v_dic='./merge_sgns_bigram_char300.txt')
# embed = lambda x: torch.randn(4, 300)

In [5]:
# DATA_PATH = '/home/featurize/data/medium/'
DATA_PATH = './dataset/medium/'
dataset = utils.ColorfulClothesBin(DATA_PATH, embed=embed, train=True)

In [6]:
resnet = models.ResNet50()
encoder_layer = nn.TransformerEncoderLayer(d_model=300, nhead=6)
clf = models.AttentionCLF(hiddens=[300, 128], encoder_layer=encoder_layer, num_layers=4)
net = models.ColorNet(resnet, clf)
img = torch.randn(2, 3, 224, 224)
tag = torch.randn(2, 4, 300)
label = torch.randint(0, 2, (2, 1)).float()
res = net((img, tag))
loss = F.binary_cross_entropy_with_logits(res, label)
print(res.dtype)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/featurize/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

torch.float32


In [11]:
val_len = int(len(dataset) * 0.1)
train, val = random_split(dataset, [len(dataset) - val_len, val_len])

In [12]:
logger = TensorBoardLogger('tb_logs', name='my_model')

In [13]:
callback = ModelCheckpoint(monitor="val_acc", mode="max", save_top_k=1, verbose=True, dirpath='tb_logs')
trainer = pl.Trainer(accelerator='gpu', devices=1, logger=logger, callbacks=[callback], max_epochs=100)
trainer.fit(net, DataLoader(train, batch_size=64, num_workers=4), DataLoader(val, batch_size=64, num_workers=4))

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/featurize/work/.local/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /home/featurize/work/ACNet/tb_logs exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type         | Params
---------------------------------------------
0 | pic_encoder | ResNet50     | 24.1 M
1 | clf         | AttentionCLF | 6.4 M 
---------------------------------------------
7.0 M     Trainable params
23.5 M    Non-trainable params
30.5 M    Total params
122.145   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.005 seconds.
Prefix dict has been built successfully.
Loading model cost 1.020 seconds.
Prefix dict has been built successfully.
Loading model cost 1.017 seconds.
Prefix dict has been built successfully.
Loading model cost 1.181 seconds.
Prefix dict has been built successfully.


Training: 0it [00:00, ?it/s]

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model from cache /tmp/jieba.cache
Loading model cost 0.745 seconds.
Prefix dict has been built successfully.
Loading model cost 0.763 seconds.
Prefix dict has been built successfully.
Loading model cost 0.827 seconds.
Loading model cost 0.835 seconds.
Prefix dict has been built successfully.
Prefix dict has been built successfully.
